In [5]:
import torch
from transformers import TimeSformerModel, TimeSformerConfig, BlipProcessor, BlipForConditionalGeneration
import cv2
import numpy as np

# Load the configuration and TimeSformer model
config = TimeSformerConfig.from_pretrained("facebook/timesformer-base-finetuned-k400")
timesformer_model = TimeSformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400")
timesformer_model.eval()

# Load the BLIP model for caption generation
processor = BlipProcessor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl")
blip_model.eval()




ImportError: cannot import name 'TimeSformerModel' from 'transformers' (/opt/anaconda3/lib/python3.11/site-packages/transformers/__init__.py)

In [ ]:
def preprocess_video(video_path, num_frames=8, frame_size=224):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_size, frame_size))
        frames.append(frame)
    cap.release()
    
    sampled_frames = np.linspace(0, len(frames) - 1, num_frames, dtype=int)
    frames = np.array([frames[i] for i in sampled_frames])
    
    frames = frames / 255.0
    frames = frames.transpose(3, 0, 1, 2)
    return torch.tensor(frames).unsqueeze(0).float()

def generate_caption(video_tensor):
    # Get embeddings from TimeSformer
    with torch.no_grad():
        video_embeddings = timesformer_model(video_tensor).last_hidden_state
    
    # Prepare embeddings for BLIP model
    video_embeddings = video_embeddings.mean(dim=1)  # Mean pool over time dimension
    inputs = processor(video_embeddings, return_tensors="pt")
    
    # Generate caption using BLIP model
    with torch.no_grad():
        outputs = blip_model.generate(**inputs)
    
    # Decode the output to text
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption

# Preprocess the video
video_tensor = preprocess_video("medical1.mp4")

# Generate caption
caption = generate_caption(video_tensor)
print(caption)